## Setting up the enviornment 

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.8.1+cu101 (Tesla T4)


## Load the data 


The dataset is created from scratch with the help of Google images by downloading the selected images under Public licence. The data is anotated with makesence.ai tool. 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q  "/content/drive/MyDrive/Custom_object_detection/train_data.zip" -d ../ 

In [4]:
!cp "/content/drive/MyDrive/Custom_object_detection/custom_data.yaml"  "/content/yolov5/data"

## set metadata
### From "/content/yolov5/data/coco128.yaml" file and create a new one according to the requirements and save it in the same folder ( /content/yolov5/data/ ) 

In [5]:
!pygmentize  "/content/yolov5/data/coco128.yaml"

# COCO 2017 dataset http://cocodataset.org - first 128 training images
# Train command: python train.py --data coco128.yaml
# Default dataset location is next to YOLOv5:
#   /parent_folder
#     /coco128
#     /yolov5


# download command/URL (optional)
download: https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../coco128/images/train2017/  # 128 images
val: ../coco128/images/train2017/  # 128 images

# number of classes
nc: 80

# class names
names: [ 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
         'skis', 'snowboard', 'sports ball', 'kite', 'baseball

In [6]:
!pygmentize "/content/yolov5/data/custom_data.yaml"

train: ../train_data/images/train/  
val: ../train_data/images/val/  

# number of classes
nc: 2

# class names
names: [  'dog', 'cow' ]


## Training

In [7]:
!pygmentize "/content/yolov5/train.py"

import argparse
import logging
import math
import os
import random
import time
from copy import deepcopy
from pathlib import Path
from threading import Thread

import numpy as np
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

import test  # import test.py to get mAP after each epoch
from models.experimental import attempt_load
from models.yolo import Model
from utils.autoanchor import check_anchors
from utils.datasets import create_dataloader
from utils.general import labels_to_class_weights, increment_path, labels_to_image_weights, init_seeds, \
    fitness, strip_optimizer, get_latest_run, check_dataset, check_file, check_git_status, check_img_size, \
    check_requirements, p

In [10]:
# Train YOLOv5s on COCO128 for 150 epochs
!python train.py --img 640 --batch 8 --epochs 150 --data custom_data.yaml --weights yolov5s.pt --nosave --cache

github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-131-g3cb9ad4 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=True, cfg='', data='./data/custom_data.yaml', device='', entity=None, epochs=150, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=True, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp2', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=8, upload_dataset=False, weights='yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-06-03 13:17:38.508294: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened

## Save the trained waights 

In [12]:
!cp "/content/yolov5/runs/train/exp2/weights/last.pt" "/content/drive/MyDrive/Custom_object_detection/"

## Inference
sample video : https://www.youtube.com/watch?v=pit4FU2lxZQ&t=47s

In [13]:
!pygmentize "/content/yolov5/detect.py"

import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path, save_one_box
from utils.plots import colors, plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized


@torch.no_grad()
def detect(opt):
    source, weights, view_img, save_txt, imgsz = opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size
    save_img = not opt.nosave and not source.endswith('.txt')  # save inference images
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    # Directories
    save_dir = increment_path(Path(opt.project) / opt.nam

In [14]:
!python detect.py --weights  "/content/drive/MyDrive/Custom_object_detection/last.pt" --img 640 --conf 0.25 --source "/content/drive/MyDrive/Custom_object_detection/input.mp4"

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/drive/MyDrive/Custom_object_detection/input.mp4', update=False, view_img=False, weights=['/content/drive/MyDrive/Custom_object_detection/last.pt'])
YOLOv5 🚀 v5.0-131-g3cb9ad4 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients, 16.3 GFLOPs
video 1/1 (1/305) /content/drive/MyDrive/Custom_object_detection/input.mp4: 384x640 2 cows, Done. (0.011s)
video 1/1 (2/305) /content/drive/MyDrive/Custom_object_detection/input.mp4: 384x640 2 cows, Done. (0.009s)
video 1/1 (3/305) /content/drive/MyDrive/Custom_object_detection/input.mp4: 384x640 2 cows, Done. (0.009s)
video 1/1 (4/305) /content/drive/MyDrive/Custom_

In [20]:
!cp "/content/yolov5/runs/detect/exp/output.mp4" "/content/drive/MyDrive/Custom_object_detection/"